<a href="https://colab.research.google.com/github/CT2020Hypernym/hyponym_and_hyperonym/blob/CNN_Maria/Maria/CNN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install spacy==2.2.3
!pip install pymorphy2

     |████████████████████████████████| 10.4MB 2.8MB/s 
     |████████████████████████████████| 3.7MB 48.6MB/s 
     |████████████████████████████████| 122kB 49.3MB/s 
     |████████████████████████████████| 2.2MB 55.6MB/s 
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9


     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 7.1MB 6.1MB/s 


In [0]:
!git clone https://github.com/buriy/spacy-ru.git

Cloning into 'spacy-ru'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 313 (delta 4), reused 7 (delta 2), pack-reused 301
Receiving objects: 100% (313/313), 117.34 MiB | 11.80 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [8]:
import spacy
import random
import pandas as pd
from spacy.util import minibatch, compounding

from spacy.lang.ru import Russian

n_texts = 7915
n_iter = 10

def dataParser():
  #maybe create new file and work with it
  data = pd.read_csv('./hyponym_and_hyperonym/DataSet/pos_pairs.csv')
  data['concat'] = data['child_context'] + ' ' + data['parent_context']
  train_pos_df = data[['concat', 'label']].dropna()
  data0 = pd.read_csv('./hyponym_and_hyperonym/DataSet/neg_pairs.csv')
  data0['concat'] = data0['child_context'] + ' ' + data0['parent_context']
  train_neg_df = data0[['concat', 'label']].dropna()
  train_df=train_pos_df.append(train_neg_df)

  train_df['tuples'] = train_df.apply(
      lambda row: (row['concat'],row['label']), axis=1)
  train = train_df['tuples'].tolist()
  return train

def load_data(limit=0, split=0.8):
    train_data = dataParser()
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{'1': bool(int(y)), '0': not bool(int(y))} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if label == '0':
                continue
            mean = 0.5
            if score >= mean and gold[label] >= mean:
                tp += 1.0
            elif score >= mean and gold[label] < mean:
                fp += 1.0
            elif score < mean and gold[label] < mean:
                tn += 1
            elif score < mean and gold[label] >= mean:
                fn += 1
    print(tp, fp, tn, fn, tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)
    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score}

if __name__ == '__main__':
    #load model (ru)
    #nlp = spacy.load('spacy-ru/ru2')
    nlp = Russian()
    nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

    if 'textcat' not in nlp.pipe_names:
      textcat = nlp.create_pipe('textcat')
      nlp.add_pipe(textcat, last=True)
    else:
      textcat = nlp.get_pipe('textcat')

    #categories
    textcat.add_label("1") #with hypernymy
    textcat.add_label("0") #without

    #load dataset

    (train_texts, train_cats), (dev_texts, dev_cats) = load_data()
    train_data = list(zip(train_texts, [{"cats": cats} for cats in train_cats]))
    print("Using {} examples ({} training, {} evaluation)"
      .format(n_texts, len(train_texts), len(dev_texts)))

    #train model
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                          losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print(
                "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
                    losses["textcat"],
                    scores["textcat_p"],
                    scores["textcat_r"],
                    scores["textcat_f"],
                )
            )
    print("Training the model ended.")

    #save the model
    output_dir = %pwd
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)


Using 7915 examples (6332 training, 1583 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
888.0 60.00000001 602.0 33.00000001 1583.0000000199998
11.067	0.937	0.964	0.950
888.0 28.00000001 634.0 33.00000001 1583.0000000199998
3.404	0.969	0.964	0.967
887.0 29.00000001 633.0 34.00000001 1583.0000000199998
2.238	0.968	0.963	0.966
883.0 27.00000001 635.0 38.00000001 1583.0000000199998
1.687	0.970	0.959	0.965
880.0 22.00000001 640.0 41.00000001 1583.0000000199998
1.257	0.976	0.955	0.965
888.0 21.00000001 641.0 33.00000001 1583.0000000199998
1.004	0.977	0.964	0.970
890.0 14.00000001 648.0 31.00000001 1583.0000000199998
0.855	0.985	0.966	0.975
893.0 17.00000001 645.0 28.00000001 1583.0000000199998
0.764	0.981	0.970	0.975
895.0 18.00000001 644.0 26.00000001 1583.0000000199998
0.520	0.980	0.972	0.976
895.0 19.00000001 643.0 26.00000001 1583.0000000199998
0.515	0.979	0.972	0.975
Training the model ended.
Saved model to /content


In [3]:
!pip install git+https://github.com/aatimofeev/spacy_russian_tokenizer.git
!git clone https://github.com/CT2020Hypernym/hyponym_and_hyperonym.git

  Cloning https://github.com/aatimofeev/spacy_russian_tokenizer.git to /tmp/pip-req-build-e3mw6_iw
  Running command git clone -q https://github.com/aatimofeev/spacy_russian_tokenizer.git /tmp/pip-req-build-e3mw6_iw
  Created wheel for spacy-russian-tokenizer: filename=spacy_russian_tokenizer-0.1.1-cp36-none-any.whl size=12674 sha256=ff4bdb3ac516493ef474cb5e3c87ccb9b2f8b1be6820da20d151043bd9a6c887
  Stored in directory: /tmp/pip-ephem-wheel-cache-dxzecbu4/wheels/37/3b/bb/cfe712f7c0b78cd08f4a2ef122d17748baf9d4bebecf2e5a54
Successfully built spacy-russian-tokenizer
Cloning into 'hyponym_and_hyperonym'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 163 (delta 12), reused 4 (delta 1), pack-reused 124
Receiving objects: 100% (163/163), 8.24 MiB | 22.99 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [0]:
!git clone https://github.com/dialogue-evaluation/taxonomy-enrichment.git

Cloning into 'taxonomy-enrichment'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 114 (delta 16), reused 33 (delta 6), pack-reused 63
Receiving objects: 100% (114/114), 17.72 MiB | 8.04 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [19]:
import spacy
import pandas as pd

nlp = spacy.load("/content")

doc = nlp('все растения имеют здоровый и ухоженный вид. Это внешняя черта, но она говорит.')
print(doc.cats)
doc = nlp('их колониального правления, это их правовая система, основанная на общем праве ,28 сентября 1998 г. за нарушение правил торговли должностное лицо было')
print(doc.cats)

doc = nlp('утка взлетела в небо. водоплавающая птица расправила крылья')
print(doc.cats)
doc = nlp('утка взлетела в небо. Это внешняя черта, но она говорит.')
print(doc.cats)

doc = nlp('все растения имеют здоровый и ухоженный вид. 28 сентября 1998 г. за нарушение правил торговли должностное лицо было')
print(doc.cats)



{'1': 0.9572031497955322, '0': 0.024976668879389763}
{'1': 0.0029625261668115854, '0': 0.9968359470367432}
{'1': 0.9911258816719055, '0': 0.006621245760470629}
{'1': 0.9786659479141235, '0': 0.014566497877240181}
{'1': 0.9118579626083374, '0': 0.057489894330501556}
